##**2 Assignment**

###**다음과 같은 CNN 모델을 작성해보자**
* Input
  * Input type: torch.Tensor
  * Input shape: (?, 1, 28, 28)
    * 여러장의, 흑백, 28x28 size의 이미지라고 가정하자
* Layers
  * Layer1
    * Conv2d >> C: 32, Kernel size (필터 크기): 3, Stride: 1, Padding: 1
    * ReLU
    * MaxPool >> Kernel size: 2, Stride: 2
    * 입-출력 (?, 1, 28, 28) >> (?, 32, 14, 14)
  * Layer2
    * Conv2d >> C: 64, Kernel size (필터 크기): 3, Stride: 1, Padding: 1
    * ReLU
    * MaxPool >> Kernel size: 2, Stride: 2
    * 입-출력 (?, 32, 14, 14) >> (?, 64, 7, 7)
  * Layer3
    * Conv2d >> C: 128, Kernel size (필터 크기): 3, Stride: 1, Padding: 1
    * ReLU
    * MaxPool >> Kernel size: 2, Stride: 2, Padding: 1
    * 입-출력 (?, 64, 7, 7) >> (?, 128, 4, 4)
  * Layer4
    * Linear >> input: 4x4x128 output: 625
    * ReLU
    * Dropout
    * 입-출력 (4x4x128) >> (625)
  * Layer5
    * Linear >> input: 625 output: 10
    * Softmax (pytorch의 Cross Entropy Loss 함수를 사용하는 것을 감안한다)

    


In [16]:
from torch import nn
import torch
import numpy as np
import torch.nn.functional as func
import torch.optim as opt

class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1, 32, 3, padding=1),
                      nn.ReLU(), nn.MaxPool2d(2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, 3, padding=1),
                      nn.ReLU(), nn.MaxPool2d(2, stride=2))
        self.layer3 = nn.Sequential(nn.Conv2d(64, 128, 3, padding=1),
                      nn.ReLU(), nn.MaxPool2d(2, stride=2, padding=1))
        self.layer4 = nn.Sequential(nn.Linear(4*4*128, 625),
                      nn.ReLU(), nn.Dropout()) #Dropout은 default값이 p=0.5
        self.linear = nn.Linear(625, 10) #layer5의 softmax는 nn.CrossEntropyLoss()를 사용

    def forward(self,x):
        in_size = x.size(0)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.reshape(in_size, -1)
        x = self.layer4(x)
        x = self.linear(x) #layer5
        return x

model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = opt.SGD(model.parameters(),lr=0.2, momentum=0.5)